## Data cleaning
### This notebook is for cleaning the house pricing data

Importing packages

In [10]:
try:
    import pandas as pd
    import plotly.express as px 
    import numpy as np
except:
    !pip install pandas
    !pip install plotly
    import pandas as pd
    import plotly.express as px 

Loading the data

In [11]:
corr_data = pd.read_pickle('../data/house_pricing_corr.pkl')

First I inspect the empty cells column wise

In [12]:
for c in corr_data.columns:
    print(f'{c}:{corr_data[c].isna().sum()}')

1stFlrSF:0
GarageArea:0
GarageYrBlt:81
GrLivArea:0
LotArea:0
LotFrontage:259
MasVnrArea:8
OpenPorchSF:0
OverallQual:0
TotalBsmtSF:0
YearBuilt:0
YearRemodAdd:0
KitchenQual:0
SalePrice:0


As LotFrontage has a low correlation with the sale price I have decided that this column can be removed, that way I presever as much as possible from the data.

In [13]:
corr_data.pop('LotFrontage')

0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64

I drop all records with empty cell in it

In [14]:
corr_data_nona=corr_data.dropna()

Defining a function to find the outlier and return with a boolean series so later I can preserve as much data as I can, since if I remove outliers in an iterating manner, it can cause a loss which is higher than necessary

In [15]:
def removeOutliers(data, col):
    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1 
    lower_range = Q1 - 1.5 * IQR
    upper_range = Q3 + 1.5 * IQR
    data1 = data[col]>lower_range
    data2 = data[col]<upper_range
    return data1 & data2

Removing the outliers

In [16]:
s = removeOutliers(corr_data_nona,corr_data_nona.columns[0])

for c in corr_data_nona.columns:
    try:
        if not c=='SalePrice' and not c=='MasVnrArea':
            t=removeOutliers(corr_data_nona,c)
            s = s & t
    except Exception as e:
        print(e)

corr_data_nona_n=corr_data_nona[s]

ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U2'), dtype('<U2')) -> None


In [17]:
corr_data_nona_n

,1stFlrSF,GarageArea,GarageYrBlt,GrLivArea,LotArea,MasVnrArea,OpenPorchSF,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,KitchenQual,SalePrice
0,856,548,2003.0,1710,8450,196.0,61,7,856,2003,2003,Gd,208500
1,1262,460,1976.0,1262,9600,0.0,0,6,1262,1976,1976,TA,181500
2,920,608,2001.0,1786,11250,162.0,42,7,920,2001,2002,Gd,223500
3,961,642,1998.0,1717,9550,0.0,35,7,756,1915,1970,Gd,140000
4,1145,836,2000.0,2198,14260,350.0,84,8,1145,2000,2000,Gd,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,953,460,1999.0,1647,7917,0.0,40,6,953,1999,2000,TA,175000
1456,2073,500,1978.0,2073,13175,119.0,0,6,1542,1978,1988,TA,210000
1457,1188,252,1941.0,2340,9042,0.0,60,7,1152,1941,2006,Gd,266500
1458,1078,240,1950.0,1078,9717,0.0,0,5,1078,1950,1996,Gd,142125


In [18]:
corr_data_nona_n.to_pickle('../data/clean_housing_data.pkl')